In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchsummary import summary
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torchvision.datasets import ImageFolder
from tqdm import notebook
from torch.utils.data import DataLoader, random_split
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
import time
import os
import copy
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report,precision_recall_curve

In [ ]:
data_dir='G:\Vinay_Canabis\Lead_II'
print(os.listdir(data_dir))

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Resize(size=(224,224)),
                transforms.Normalize((0.3433, 0.7419, 0.5770), (0.3453, 0.0661, 0.2652))
                ])

In [ ]:
dataset=ImageFolder(data_dir,transform=transform)
torch.manual_seed(42)
train_ds, val_ds, test_ds = random_split(dataset, [0.8,0.15,0.05])
len(train_ds),len(val_ds),len(test_ds)

In [ ]:
trainloader =DataLoader(train_ds, batch_size=16, shuffle=True,num_workers=4)
valloader = DataLoader(val_ds, batch_size=32, shuffle=True,num_workers=4)
testloader = DataLoader(test_ds, batch_size=32, shuffle=False)

In [ ]:
def evaluation(dataloader, model):
    total, correct = 0, 0
    model.eval()
    for data in dataloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, pred = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (pred == labels).sum().item()
    return 100 * correct / total

In [ ]:
def train_func(model, loss_fn, opt, trainloader, valloader, scheduler , epochs):
    loss_epoch_arr = []
    train_accuracy_lis = []
    val_accuracy_lis = []
    val_loss_epoch_arr = []

    max_epochs = epochs
    min_loss = 1000
    model.train()

    for epoch in notebook.tqdm(range(max_epochs), total=max_epochs, unit="epochs"):
        model.train()
        for i, data in enumerate(trainloader, 0):
    
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            opt.zero_grad()
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            loss.backward()
            opt.step()
            scheduler.step()

            if min_loss > loss.item():
                min_loss = loss.item()
                best_model = copy.deepcopy(model.state_dict())
                print("Min loss %0.2f" %min_loss)
            del inputs, labels, outputs
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            if i % 2000 == 0:
                print('Iteration: %d, Loss: %0.2f' % (i, loss.item()))
            
       
        loss_epoch_arr.append(loss.item())
        train_accuracy = evaluation(trainloader, model)
        val_accuracy = evaluation(valloader, model)
        print('Epoch: %d/%d, Test acc: %0.2f, Train acc: %0.2f' % (
              epoch, max_epochs, val_accuracy, train_accuracy))
        train_accuracy_lis.append(train_accuracy)
        val_accuracy_lis.append(val_accuracy)
    
        model.eval()
        with torch.no_grad():
            for i, data in enumerate(valloader, 0):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = loss_fn(outputs, labels)

        del inputs, labels, outputs
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

        val_loss_epoch_arr.append(loss.item())
    return best_model, loss_epoch_arr, train_accuracy_lis, val_accuracy_lis, val_loss_epoch_arr

In [ ]:
def generate_classification_report(model, dataloader, device, filepath):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for data in dataloader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.append(preds)
            all_labels.append(labels)

    all_preds = torch.cat(all_preds).cpu().numpy()
    all_labels = torch.cat(all_labels).cpu().numpy()

    report = classification_report(all_labels, all_preds, target_names=[str(i) for i in range(len(set(all_labels)))])

    # Save the classification report to a text file
    with open(filepath, 'w') as f:
        f.write(report)
    
    print(f"Classification report saved to '{filepath}'")

In [ ]:
def plot_evaluation_metrics(model, dataloader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for data in dataloader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.append(preds)
            all_labels.append(labels)

    all_preds = torch.cat(all_preds).cpu().numpy()
    all_labels = torch.cat(all_labels).cpu().numpy()
    return all_preds, all_labels

In [ ]:
num_epochs = 50
epochs = list(range(1, num_epochs + 1))

## Resnet 50

In [ ]:
resnet = models.resnet50(weights=True)

In [ ]:
for param in resnet.parameters():
    param.requires_grad=True

In [ ]:
num_features = resnet.fc.in_features
new_classifier = nn.Sequential(
    nn.Linear(num_features, 256), nn.ReLU(), nn.Dropout(0.2),
    nn.Linear(256, 2))
resnet.fc = new_classifier

In [ ]:
resnet =resnet.to(device)
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(resnet.parameters(), lr=0.0001)
summary(resnet, (3, 224, 224))

In [ ]:
steps_per_epoch = len(trainloader)

# Set the maximum learning rate and the factor by which to increase it
max_lr = 0.01
lr_step_factor = 8

# Define the learning rate scheduler with the OneCycleLR policy
scheduler = lr_scheduler.OneCycleLR(opt, max_lr, total_steps=50* steps_per_epoch, pct_start=0.3, div_factor=lr_step_factor)


In [ ]:
best_model, loss_epoch_arr, train_accuracy_lis, val_accuracy_lis, val_loss_epoch_arr = train_func(resnet, loss_fn, opt, trainloader, valloader, scheduler , epochs=50
                                                                            )      

In [ ]:
evaluation(testloader,resnet)

In [ ]:
folder_name = 'Resnet_50_results'
os.makedirs(folder_name, exist_ok=True)

In [ ]:
data = {
    'Loss': loss_epoch_arr,
    'Train Accuracy': train_accuracy_lis,
    'Validation Accuracy': val_accuracy_lis,
    'Validation Loss': val_loss_epoch_arr
}
df = pd.DataFrame(data)

# Save to CSV file
df.to_csv('Resnet_50_results/resnet50_OneCLR_training_data.csv', index=False)

In [ ]:
sns.set(style='darkgrid') 
plt.figure(figsize=(12, 6))
sns.lineplot(x = epochs,y = train_accuracy_lis, label='Training accuracy', marker='o', color='green')
sns.lineplot(x = epochs,y = val_accuracy_lis, label='Validation accuracy', marker='o', color='red')
plt.title('Resnet_50 Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig(os.path.join(folder_name, 'Resnet50_accuracy_plot.png'))
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.lineplot(x = epochs,y = loss_epoch_arr, label='Training loss', marker='o', color='green')
sns.lineplot(x = epochs, y = val_loss_epoch_arr, label='Validation loss', marker='o', color='red')
plt.title('Resnet_50 Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig(os.path.join(folder_name, 'Resnet_50_loss_plot.png'))
plt.show()

In [ ]:
all_preds, all_labels = plot_evaluation_metrics(resnet,valloader,device)

In [ ]:
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d',cmap = "magma")
plt.title('Resnet_50 Val_Confusion_Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.savefig(os.path.join(folder_name, 'Resnet_50_Val_Confusion_Matrix.png'))
plt.show()

In [ ]:
generate_classification_report(resnet,testloader, device, 'Resnet_50_results/Val_report.txt')

In [ ]:
all_preds, all_labels = plot_evaluation_metrics(resnet,testloader,device)

In [ ]:
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d',cmap = "magma")
plt.title('Resnet_50 Test_Confusion_Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.savefig(os.path.join(folder_name, 'Resnet_50_Test_Confusion_Matrix.png'))
plt.show()

In [ ]:
generate_classification_report(resnet,testloader, device, 'Resnet_50_results/Test_report.txt')

## Resnet_152

In [ ]:
resnet_152 = models.resnet152(weights=True)

In [ ]:
for param in resnet_152.parameters():
    param.requires_grad=True

In [ ]:
num_features = resnet_152.fc.in_features
new_classifier = nn.Sequential(
    nn.Linear(num_features, 256), nn.ReLU(), nn.Dropout(0.2),
    nn.Linear(256, 2))
resnet_152.fc = new_classifier

In [ ]:
resnet_152 =resnet_152.to(device)
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(resnet_152.parameters(), lr=0.0001)
summary(resnet_152, (3, 224, 224))

In [ ]:
steps_per_epoch = len(trainloader)

# Set the maximum learning rate and the factor by which to increase it
max_lr = 0.01
lr_step_factor = 8

# Define the learning rate scheduler with the OneCycleLR policy
scheduler = lr_scheduler.OneCycleLR(opt, max_lr, total_steps=50* steps_per_epoch, pct_start=0.3, div_factor=lr_step_factor)

In [ ]:
best_model, loss_epoch_arr, train_accuracy_lis, val_accuracy_lis, val_loss_epoch_arr = train_func(resnet_152, loss_fn, opt, trainloader, valloader, scheduler , epochs=60
                                                                            )      

In [ ]:
evaluation(testloader,resnet_152)

In [ ]:
folder_name = 'Resnet_152_results'
os.makedirs(folder_name, exist_ok=True)

In [ ]:
data = {
    'Loss': loss_epoch_arr,
    'Train Accuracy': train_accuracy_lis,
    'Validation Accuracy': val_accuracy_lis,
    'Validation Loss': val_loss_epoch_arr
}
df = pd.DataFrame(data)

# Save to CSV file
df.to_csv('Resnet_152_results/resnet152_OneCLR_training_data.csv', index=False)

In [ ]:
sns.set(style='darkgrid') 
plt.figure(figsize=(12, 6))
sns.lineplot(x = epochs,y = train_accuracy_lis, label='Training accuracy', marker='o', color='green')
sns.lineplot(x = epochs,y = val_accuracy_lis, label='Validation accuracy', marker='o', color='red')
plt.title('Resnet_152 Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig(os.path.join(folder_name, 'Resnet_152_accuracy_plot.png'))
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.lineplot(x = epochs,y = loss_epoch_arr, label='Training loss', marker='o', color='green')
sns.lineplot(x = epochs, y = val_loss_epoch_arr, label='Validation loss', marker='o', color='red')
plt.title('Resnet_152 Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig(os.path.join(folder_name, 'Resnet_152_loss_plot.png'))
plt.show()

In [ ]:
all_preds, all_labels = plot_evaluation_metrics(resnet_152,valloader,device)

In [ ]:
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d',cmap = "magma")
plt.title('Resnet_152 Val_Confusion_Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.savefig(os.path.join(folder_name, 'Resnet_152_Val_Confusion_Matrix.png'))
plt.show()

In [ ]:
generate_classification_report(resnet_152,testloader, device, 'Resnet_152_results/Val_report.txt')

In [ ]:
all_preds, all_labels = plot_evaluation_metrics(resnet_152,testloader,device)

In [ ]:
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d',cmap = "magma")
plt.title('Resnet_152 Test_Confusion_Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.savefig(os.path.join(folder_name, 'Resnet_152_Test_Confusion_Matrix.png'))
plt.show()

In [ ]:
generate_classification_report(resnet_152,testloader, device, 'Resnet_152_results/Test_report.txt')

## Resnet 18

In [ ]:
resnet_18 = models.resnet18(weights=True)

In [ ]:
for param in resnet_18.parameters():
    param.requires_grad=True

In [ ]:
num_features = resnet_18.fc.in_features
new_classifier = nn.Sequential(
    nn.Linear(num_features, 256), nn.ReLU(), nn.Dropout(0.2),
    nn.Linear(256, 2))
resnet_18.fc = new_classifier

In [ ]:
resnet_18 =resnet_18.to(device)
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(resnet_18.parameters(), lr=0.0001)
summary(resnet_18, (3, 224, 224))

In [ ]:
steps_per_epoch = len(trainloader)

# Set the maximum learning rate and the factor by which to increase it
max_lr = 0.01
lr_step_factor = 8

# Define the learning rate scheduler with the OneCycleLR policy
scheduler = lr_scheduler.OneCycleLR(opt, max_lr, total_steps=50* steps_per_epoch, pct_start=0.3, div_factor=lr_step_factor)

In [ ]:
best_model, loss_epoch_arr, train_accuracy_lis, val_accuracy_lis, val_loss_epoch_arr = train_func(resnet_18, loss_fn, opt, trainloader, valloader, scheduler , epochs=60
                                                                            )      

In [ ]:
evaluation(testloader,resnet_18)

In [ ]:
folder_name = 'Resnet_18_results'
os.makedirs(folder_name, exist_ok=True)

In [ ]:
data = {
    'Loss': loss_epoch_arr,
    'Train Accuracy': train_accuracy_lis,
    'Validation Accuracy': val_accuracy_lis,
    'Validation Loss': val_loss_epoch_arr
}
df = pd.DataFrame(data)

# Save to CSV file
df.to_csv('Resnet_18_results/resnet18_OneCLR_training_data.csv', index=False)

In [ ]:
sns.set(style='darkgrid') 
plt.figure(figsize=(12, 6))
sns.lineplot(x = epochs,y = train_accuracy_lis, label='Training accuracy', marker='o', color='green')
sns.lineplot(x = epochs,y = val_accuracy_lis, label='Validation accuracy', marker='o', color='red')
plt.title('Resnet_18 Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig(os.path.join(folder_name, 'Resnet_18_accuracy_plot.png'))
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.lineplot(x = epochs,y = loss_epoch_arr, label='Training loss', marker='o', color='green')
sns.lineplot(x = epochs, y = val_loss_epoch_arr, label='Validation loss', marker='o', color='red')
plt.title('Resnet_18 Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig(os.path.join(folder_name, 'Resnet_18_loss_plot.png'))
plt.show()

In [ ]:
all_preds, all_labels = plot_evaluation_metrics(resnet_18,valloader,device)

In [ ]:
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d',cmap = "magma")
plt.title('Resnet_18 Val_Confusion_Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.savefig(os.path.join(folder_name, 'Resnet_18_Val_Confusion_Matrix.png'))
plt.show()

In [ ]:
generate_classification_report(resnet_18, testloader, device, 'Resnet_18_results/Val_report.txt')

In [ ]:
all_preds, all_labels = plot_evaluation_metrics(resnet_18, testloader,device)

In [ ]:
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d',cmap = "magma")
plt.title('Resnet_18 Test_Confusion_Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.savefig(os.path.join(folder_name, 'Resnet_18_Test_Confusion_Matrix.png'))
plt.show()

In [ ]:
generate_classification_report(resnet_18,testloader, device, 'Resnet_18_results/Test_report.txt')

## Resnet 34

In [ ]:
resnet_34 = models.resnet34(weights=True)

In [ ]:
for param in resnet_34.parameters():
    param.requires_grad=True

In [ ]:
num_features = resnet_34.fc.in_features
new_classifier = nn.Sequential(
    nn.Linear(num_features, 256), nn.ReLU(), nn.Dropout(0.2),
    nn.Linear(256, 2))
resnet_34.fc = new_classifier

In [ ]:
resnet_34 =resnet_34.to(device)
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(resnet_34.parameters(), lr=0.0001)
summary(resnet_34, (3, 224, 224))

In [ ]:
steps_per_epoch = len(trainloader)

# Set the maximum learning rate and the factor by which to increase it
max_lr = 0.01
lr_step_factor = 8

# Define the learning rate scheduler with the OneCycleLR policy
scheduler = lr_scheduler.OneCycleLR(opt, max_lr, total_steps=50* steps_per_epoch, pct_start=0.3, div_factor=lr_step_factor)

In [ ]:
best_model, loss_epoch_arr, train_accuracy_lis, val_accuracy_lis, val_loss_epoch_arr = train_func(resnet_34, loss_fn, opt, trainloader, valloader, scheduler , epochs=60
                                                                            )      

In [ ]:
evaluation(testloader,resnet_34)

In [ ]:
folder_name = 'Resnet_34_results'
os.makedirs(folder_name, exist_ok=True)

In [ ]:
data = {
    'Loss': loss_epoch_arr,
    'Train Accuracy': train_accuracy_lis,
    'Validation Accuracy': val_accuracy_lis,
    'Validation Loss': val_loss_epoch_arr
}
df = pd.DataFrame(data)

# Save to CSV file
df.to_csv('Resnet_34_results/resnet34_OneCLR_training_data.csv', index=False)

In [ ]:
sns.set(style='darkgrid') 
plt.figure(figsize=(12, 6))
sns.lineplot(x = epochs,y = train_accuracy_lis, label='Training accuracy', marker='o', color='green')
sns.lineplot(x = epochs,y = val_accuracy_lis, label='Validation accuracy', marker='o', color='red')
plt.title('Resnet_34 Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig(os.path.join(folder_name, 'Resnet_34_accuracy_plot.png'))
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.lineplot(x = epochs,y = loss_epoch_arr, label='Training loss', marker='o', color='green')
sns.lineplot(x = epochs, y = val_loss_epoch_arr, label='Validation loss', marker='o', color='red')
plt.title('Resnet_34 Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig(os.path.join(folder_name, 'Resnet_34_loss_plot.png'))
plt.show()

In [ ]:
all_preds, all_labels = plot_evaluation_metrics(resnet_18,valloader,device)

In [ ]:
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d',cmap = "magma")
plt.title('Resnet_34 Val_Confusion_Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.savefig(os.path.join(folder_name, 'Resnet_34_Val_Confusion_Matrix.png'))
plt.show()

In [ ]:
generate_classification_report(resnet_34, testloader, device, 'Resnet_34_results/Val_report.txt')

In [ ]:
all_preds, all_labels = plot_evaluation_metrics(resnet_34, testloader,device)

In [ ]:
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d',cmap = "magma")
plt.title('Resnet_34 Test_Confusion_Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.savefig(os.path.join(folder_name, 'Resnet_34_Test_Confusion_Matrix.png'))
plt.show()

In [ ]:
generate_classification_report(resnet_34,testloader, device, 'Resnet_34_results/Test_report.txt')

## Resnet 101

In [ ]:
resnet_101 = models.resnet101(weights=True)

In [ ]:
for param in resnet_101.parameters():
    param.requires_grad=True

In [ ]:
num_features = resnet_101.fc.in_features
new_classifier = nn.Sequential(
    nn.Linear(num_features, 256), nn.ReLU(), nn.Dropout(0.2),
    nn.Linear(256, 2))
resnet_101.fc = new_classifier

In [ ]:
resnet_101 =resnet_101.to(device)
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(resnet_101.parameters(), lr=0.0001)
summary(resnet_101, (3, 224, 224))

In [ ]:
steps_per_epoch = len(trainloader)

# Set the maximum learning rate and the factor by which to increase it
max_lr = 0.01
lr_step_factor = 8

# Define the learning rate scheduler with the OneCycleLR policy
scheduler = lr_scheduler.OneCycleLR(opt, max_lr, total_steps=50* steps_per_epoch, pct_start=0.3, div_factor=lr_step_factor)

In [ ]:
best_model, loss_epoch_arr, train_accuracy_lis, val_accuracy_lis, val_loss_epoch_arr = train_func(resnet_101, loss_fn, opt, trainloader, valloader, scheduler , epochs=60
                                                                            )      

In [ ]:
evaluation(testloader,resnet_101)

In [ ]:
folder_name = 'Resnet_101_results'
os.makedirs(folder_name, exist_ok=True)

In [ ]:
data = {
    'Loss': loss_epoch_arr,
    'Train Accuracy': train_accuracy_lis,
    'Validation Accuracy': val_accuracy_lis,
    'Validation Loss': val_loss_epoch_arr
}
df = pd.DataFrame(data)

# Save to CSV file
df.to_csv('Resnet_101_results/resnet101_OneCLR_training_data.csv', index=False)

In [ ]:
sns.set(style='darkgrid') 
plt.figure(figsize=(12, 6))
sns.lineplot(x = epochs,y = train_accuracy_lis, label='Training accuracy', marker='o', color='green')
sns.lineplot(x = epochs,y = val_accuracy_lis, label='Validation accuracy', marker='o', color='red')
plt.title('Resnet_101 Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig(os.path.join(folder_name, 'Resnet_101_accuracy_plot.png'))
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.lineplot(x = epochs,y = loss_epoch_arr, label='Training loss', marker='o', color='green')
sns.lineplot(x = epochs, y = val_loss_epoch_arr, label='Validation loss', marker='o', color='red')
plt.title('Resnet_101 Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig(os.path.join(folder_name, 'Resnet_101_loss_plot.png'))
plt.show()

In [ ]:
all_preds, all_labels = plot_evaluation_metrics(resnet_101,valloader,device)

In [ ]:
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d',cmap = "magma")
plt.title('Resnet_101 Val_Confusion_Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.savefig(os.path.join(folder_name, 'Resnet_101_Val_Confusion_Matrix.png'))
plt.show()

In [ ]:
generate_classification_report(resnet_101, testloader, device, 'Resnet_101_results/Val_report.txt')

In [ ]:
all_preds, all_labels = plot_evaluation_metrics(resnet_101, testloader,device)

In [ ]:
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d',cmap = "magma")
plt.title('Resnet_101 Test_Confusion_Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.savefig(os.path.join(folder_name, 'Resnet_101_Test_Confusion_Matrix.png'))
plt.show()

In [ ]:
generate_classification_report(resnet_101,testloader, device, 'Resnet_101_results/Test_report.txt')

## VGG16

In [ ]:
vgg16 = models.vgg16_bn(weights=True)

In [ ]:
for param in vgg16.features.parameters():
    param.requires_grad = True

In [ ]:
num_features = vgg16.classifier[6].in_features
vgg16.classifier[6] = nn.Linear(num_features, 2)

In [ ]:
vgg16 = vgg16.to(device)
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(vgg16.parameters(), lr=0.0001)

In [ ]:
steps_per_epoch = len(trainloader)

# Set the maximum learning rate and the factor by which to increase it
max_lr = 0.01
lr_step_factor = 8

# Define the learning rate scheduler with the OneCycleLR policy
scheduler = lr_scheduler.OneCycleLR(opt, max_lr, total_steps=50* steps_per_epoch, pct_start=0.3, div_factor=lr_step_factor)

In [ ]:
best_model, loss_epoch_arr, train_accuracy_lis, val_accuracy_lis, val_loss_epoch_arr = train_func(vgg16, loss_fn, opt, trainloader, valloader, scheduler , epochs=60
                                                                            )      

In [ ]:
evaluation(testloader,vgg16)

In [ ]:
folder_name = 'vgg_16_results'
os.makedirs(folder_name, exist_ok=True)

In [ ]:
data = {
    'Loss': loss_epoch_arr,
    'Train Accuracy': train_accuracy_lis,
    'Validation Accuracy': val_accuracy_lis,
    'Validation Loss': val_loss_epoch_arr
}
df = pd.DataFrame(data)

# Save to CSV file
df.to_csv('vgg_16_results/vgg16_OneCLR_training_data.csv', index=False)

In [ ]:
sns.set(style='darkgrid') 
plt.figure(figsize=(12, 6))
sns.lineplot(x = epochs,y = train_accuracy_lis, label='Training accuracy', marker='o', color='green')
sns.lineplot(x = epochs,y = val_accuracy_lis, label='Validation accuracy', marker='o', color='red')
plt.title('vgg_16 Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig(os.path.join(folder_name, 'vgg_16_accuracy_plot.png'))
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.lineplot(x = epochs,y = loss_epoch_arr, label='Training loss', marker='o', color='green')
sns.lineplot(x = epochs, y = val_loss_epoch_arr, label='Validation loss', marker='o', color='red')
plt.title('vgg_16 Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig(os.path.join(folder_name, 'vgg_16_loss_plot.png'))
plt.show()

In [ ]:
all_preds, all_labels = plot_evaluation_metrics(vgg16,valloader,device)

In [ ]:
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d',cmap = "magma")
plt.title('vgg_16 Val_Confusion_Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.savefig(os.path.join(folder_name, 'vgg_16_Val_Confusion_Matrix.png'))
plt.show()

In [ ]:
generate_classification_report(vgg16, testloader, device, 'vgg_16_results/Val_report.txt')

In [ ]:
all_preds, all_labels = plot_evaluation_metrics(vgg16, testloader,device)

In [ ]:
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d',cmap = "magma")
plt.title('vgg_16 Test_Confusion_Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.savefig(os.path.join(folder_name, 'vgg_16_Test_Confusion_Matrix.png'))
plt.show()

In [ ]:
generate_classification_report(vgg_16,testloader, device, 'vgg_16_results/Test_report.txt')

## VGG19

In [ ]:
vgg19= models.vgg19_bn(weights=True)

In [ ]:
for param in vgg19.features.parameters():
    param.requires_grad = True

In [ ]:
num_features = vgg19.classifier[6].in_features
vgg19.classifier[6] = nn.Linear(num_features, 2)

In [ ]:
vgg19 = vgg19.to(device)
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(vgg19.parameters(), lr=0.0001)

In [ ]:
steps_per_epoch = len(trainloader)

# Set the maximum learning rate and the factor by which to increase it
max_lr = 0.01
lr_step_factor = 8

# Define the learning rate scheduler with the OneCycleLR policy
scheduler = lr_scheduler.OneCycleLR(opt, max_lr, total_steps=50* steps_per_epoch, pct_start=0.3, div_factor=lr_step_factor)

In [ ]:
best_model, loss_epoch_arr, train_accuracy_lis, val_accuracy_lis, val_loss_epoch_arr = train_func(vgg19, loss_fn, opt, trainloader, valloader, scheduler , epochs=60
                                                                            )      

In [ ]:
evaluation(testloader,vgg19)

In [ ]:
folder_name = 'vgg_19_results'
os.makedirs(folder_name, exist_ok=True)

In [ ]:
data = {
    'Loss': loss_epoch_arr,
    'Train Accuracy': train_accuracy_lis,
    'Validation Accuracy': val_accuracy_lis,
    'Validation Loss': val_loss_epoch_arr
}
df = pd.DataFrame(data)

# Save to CSV file
df.to_csv('vgg_19_results/vgg19_OneCLR_training_data.csv', index=False)

In [ ]:
sns.set(style='darkgrid') 
plt.figure(figsize=(12, 6))
sns.lineplot(x = epochs,y = train_accuracy_lis, label='Training accuracy', marker='o', color='green')
sns.lineplot(x = epochs,y = val_accuracy_lis, label='Validation accuracy', marker='o', color='red')
plt.title('vgg_19 Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig(os.path.join(folder_name, 'vgg_19_accuracy_plot.png'))
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.lineplot(x = epochs,y = loss_epoch_arr, label='Training loss', marker='o', color='green')
sns.lineplot(x = epochs, y = val_loss_epoch_arr, label='Validation loss', marker='o', color='red')
plt.title('vgg_19 Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig(os.path.join(folder_name, 'vgg_19_loss_plot.png'))
plt.show()

In [ ]:
all_preds, all_labels = plot_evaluation_metrics(vgg19,valloader,device)

In [ ]:
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d',cmap = "magma")
plt.title('vgg_19 Val_Confusion_Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.savefig(os.path.join(folder_name, 'vgg_19_Val_Confusion_Matrix.png'))
plt.show()

In [ ]:
generate_classification_report(vgg19, testloader, device, 'vgg_19_results/Val_report.txt')

In [ ]:
all_preds, all_labels = plot_evaluation_metrics(vgg19, testloader,device)

In [ ]:
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d',cmap = "magma")
plt.title('vgg_19 Test_Confusion_Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.savefig(os.path.join(folder_name, 'vgg_19_Test_Confusion_Matrix.png'))
plt.show()

In [ ]:
generate_classification_report(vgg_19,testloader, device, 'vgg_19_results/Test_report.txt')

## Densenet121

In [ ]:
densenet121 = models.densenet121(pretrained=True)

In [ ]:
for param in densenet121.features.parameters():
    param.requires_grad = True


In [ ]:
num_features = densenet121.classifier.in_features
densenet121.classifier = nn.Linear(num_features, 2)

In [ ]:
densenet121 = densenet121.to(device)
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(densenet121.parameters(), lr=0.0001)

In [ ]:
steps_per_epoch = len(trainloader)

# Set the maximum learning rate and the factor by which to increase it
max_lr = 0.01
lr_step_factor = 8

# Define the learning rate scheduler with the OneCycleLR policy
scheduler = lr_scheduler.OneCycleLR(opt, max_lr, total_steps=50* steps_per_epoch, pct_start=0.3, div_factor=lr_step_factor)

In [ ]:
best_model, loss_epoch_arr, train_accuracy_lis, val_accuracy_lis, val_loss_epoch_arr = train_func(densenet121, loss_fn, opt, trainloader, valloader, scheduler , epochs=60
                                                                            )      

In [ ]:
evaluation(testloader,densenet121)

In [ ]:
folder_name = 'densenet_121_results'
os.makedirs(folder_name, exist_ok=True)

In [ ]:
data = {
    'Loss': loss_epoch_arr,
    'Train Accuracy': train_accuracy_lis,
    'Validation Accuracy': val_accuracy_lis,
    'Validation Loss': val_loss_epoch_arr
}
df = pd.DataFrame(data)

# Save to CSV file
df.to_csv('densenet_121_results/densenet_121_OneCLR_training_data.csv', index=False)

In [ ]:
sns.set(style='darkgrid') 
plt.figure(figsize=(12, 6))
sns.lineplot(x = epochs,y = train_accuracy_lis, label='Training accuracy', marker='o', color='green')
sns.lineplot(x = epochs,y = val_accuracy_lis, label='Validation accuracy', marker='o', color='red')
plt.title('densenet_121 Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig(os.path.join(folder_name, 'densenet_121_accuracy_plot.png'))
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.lineplot(x = epochs,y = loss_epoch_arr, label='Training loss', marker='o', color='green')
sns.lineplot(x = epochs, y = val_loss_epoch_arr, label='Validation loss', marker='o', color='red')
plt.title('densenet_121 Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig(os.path.join(folder_name, 'densenet_121_loss_plot.png'))
plt.show()

In [ ]:
all_preds, all_labels = plot_evaluation_metrics(densenet121,valloader,device)

In [ ]:
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d',cmap = "magma")
plt.title('densenet_121 Val_Confusion_Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.savefig(os.path.join(folder_name, 'densenet_121_Val_Confusion_Matrix.png'))
plt.show()

In [ ]:
generate_classification_report(densenet121, testloader, device, 'densenet_121_results/Val_report.txt')

In [ ]:
all_preds, all_labels = plot_evaluation_metrics(densenet121, testloader,device)

In [ ]:
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d',cmap = "magma")
plt.title('densenet_121 Test_Confusion_Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.savefig(os.path.join(folder_name, 'densenet_121_Test_Confusion_Matrix.png'))
plt.show()

In [ ]:
generate_classification_report(densenet121,testloader, device, 'densenet_121_results/Test_report.txt')

## DenseNet201

In [ ]:
densenet201 = models.densenet201(pretrained=True)

In [ ]:
for param in densenet201.features.parameters():
    param.requires_grad = True


In [ ]:
num_features = densenet201.classifier.in_features
densenet201.classifier = nn.Linear(num_features, 2)

In [ ]:
densenet201 = densenet201.to(device)
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(densenet201.parameters(), lr=0.0001)

In [ ]:
steps_per_epoch = len(trainloader)

# Set the maximum learning rate and the factor by which to increase it
max_lr = 0.01
lr_step_factor = 8

# Define the learning rate scheduler with the OneCycleLR policy
scheduler = lr_scheduler.OneCycleLR(opt, max_lr, total_steps=50* steps_per_epoch, pct_start=0.3, div_factor=lr_step_factor)

In [ ]:
best_model, loss_epoch_arr, train_accuracy_lis, val_accuracy_lis, val_loss_epoch_arr = train_func(densenet201, loss_fn, opt, trainloader, valloader, scheduler , epochs=60
                                                                            )      

In [ ]:
evaluation(testloader,densenet201)

In [ ]:
folder_name = 'densenet_201_results'
os.makedirs(folder_name, exist_ok=True)

In [ ]:
data = {
    'Loss': loss_epoch_arr,
    'Train Accuracy': train_accuracy_lis,
    'Validation Accuracy': val_accuracy_lis,
    'Validation Loss': val_loss_epoch_arr
}
df = pd.DataFrame(data)

# Save to CSV file
df.to_csv('densenet_201_results/densenet_201_OneCLR_training_data.csv', index=False)

In [ ]:
sns.set(style='darkgrid') 
plt.figure(figsize=(12, 6))
sns.lineplot(x = epochs,y = train_accuracy_lis, label='Training accuracy', marker='o', color='green')
sns.lineplot(x = epochs,y = val_accuracy_lis, label='Validation accuracy', marker='o', color='red')
plt.title('densenet_201 Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig(os.path.join(folder_name, 'densenet_201_accuracy_plot.png'))
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.lineplot(x = epochs,y = loss_epoch_arr, label='Training loss', marker='o', color='green')
sns.lineplot(x = epochs, y = val_loss_epoch_arr, label='Validation loss', marker='o', color='red')
plt.title('densenet_201 Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig(os.path.join(folder_name, 'densenet_201_loss_plot.png'))
plt.show()

In [ ]:
all_preds, all_labels = plot_evaluation_metrics(densenet201,valloader,device)

In [ ]:
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d',cmap = "magma")
plt.title('densenet_201 Val_Confusion_Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.savefig(os.path.join(folder_name, 'densenet_201_Val_Confusion_Matrix.png'))
plt.show()

In [ ]:
generate_classification_report(densenet201, testloader, device, 'densenet_201_results/Val_report.txt')

In [ ]:
all_preds, all_labels = plot_evaluation_metrics(densenet201, testloader,device)

In [ ]:
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d',cmap = "magma")
plt.title('densenet_201 Test_Confusion_Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.savefig(os.path.join(folder_name, 'densenet_201_Test_Confusion_Matrix.png'))
plt.show()

In [ ]:
generate_classification_report(densenet201,testloader, device, 'densenet_201_results/Test_report.txt')